In [74]:
import numpy as np
from scipy.stats import entropy

# <font color = 'red'> ЛР 6. Дерево решений. Метрики f1 качества классификации. </font>

Сложность: <font color = 'green'> Легко  </font>.

Дата составления: 07.10.2024

Срок выполнения: 2 недели (с момента первой практики после выдачи).

Автор: ст. преподаватель Кушнеров А.В.

## <font color = 'green'> 1. Энтропия.  </font>

**Энтропия Шеннона** - мера хаотичности множества. Пусть задано множество из $N$ представителей $s$ различных классов. $X = (x_{1},x_{1},x_{1},...,x_{2},x_{2},...,...x_{s},x_{s},...)$.

Энтропия тогда может быть вычислена по формуле:

$$S(X)=-\displaystyle\sum_{i=1}^{s} p_{i}\log_2 p_{i}$$

Где, $p_{i}=N_{i}/N$.

<font color = 'red' size = 5>Задание 1 </font>

Реализуйте функцию для подсчёта энтропии Шеннонна для заданной выборки. Сравните результат работы со встроенной функцией.


[Справочная информация](https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%8D%D0%BD%D1%82%D1%80%D0%BE%D0%BF%D0%B8%D1%8F)


In [75]:
test = np.random.randint(0,3,100)
test

unique, counts = np.unique(test, return_counts=True)

In [389]:
entropy(np.array([1,1,1]),base=2)

1.584962500721156

In [395]:
#task 1
def my_entropy(x, base = 1):
  _, counts = np.unique(x, return_counts=True)
  return -((counts / x.shape[0]) * np.log2(counts / x.shape[0])).sum()



my_entropy(np.array([1,1,1]))

-0.0

0

## <font color = 'green'> 2. Прирост информации и дерево принятия решений  </font>

Пусть исходное множество $X$ было разделено на несколько подмножеств (чаще всего на 2). В каждом из них можем посчитать энтропию. Мы ожидаем, что наше разбиение уменьшит энтропию в каждом из подмножеств или в целом, что должно приблизить нас к решению задачи классификации. Для того, чтобы получить полную картину вводят понятие прироста информации.

$$IG(Q) = S_{0}-\displaystyle\sum_{i=1}^{q} \frac{N_{i}}{N}S_{i}$$

Тут имеем: $S_{0}$ - энтропия исходного множества, $N_{i}$ - количесвто элементов в каждом новом классе после разбиения, $N$ - исходное количество элементов, $q$ - количество множеств после разбиения,  $S_{i}$ - энтропия новых множеств.


Суть работы решающего дерева в разделении обучающего множества на подмножества, так чтобы энтропия выборки меток уменьшалась по каждому подмножеству, а прирост информации увеличивался. В качестве критерия для разделения(по признакам) выбирают тот, который даёт наибольший прирост информации (по меткам). Эта процедура повторяется рекурсивно.

[Справочная информация](https://scikit-learn.org/stable/modules/tree.html#tree)

<font color = 'red' size = 5>Задание 2 </font>


1. Реализуйте учебное приложение, которое строит решающее дерево на основе энтропии классификации на данных состоящих не более чем из двух признаков.
2. Проверьте своё приложение на простых искусственных данных и сравните со встроенным классом DecisionTreeClassifier.
3. Реализуйте учебное приложение, которое строит решающее дерево на основе критерия Джини классификации на данных состоящих не более чем из двух признаков.*

In [68]:
sorted([(1,2,3,4),(4,3,2,1),(4,5,6,7)], key = lambda x: x[3], reverse = True)

[(4, 5, 6, 7), (1, 2, 3, 4), (4, 3, 2, 1)]

In [375]:
from collections import Counter

class DisicionTree:
  def __init__(self, x, y, max_depth=3):
    self.x, self.y = x, y
    self.root = self.Leaf(x, y, None)
    self.leafs = [self.root]
    self.max_depth = max_depth

  def information_gain(self, y, y_split):
    return entropy(y,base = 2) - (np.array([entropy(yi, base=2) for yi in y_split]) * np.array([len(yi) for yi in y_split])).sum() / len(y)


  def predict(self, x):
    return self.root.predict(x)

  def split_one_leaf(self):
    leaf_stats = [(leaf,*self.best_split(leaf.x, leaf.y)) for leaf in self.leafs if leaf.depth <= self.max_depth]
    leaf, feature, condition, _ = sorted(leaf_stats, key = lambda x: x[-1], reverse=True)[0]
    self.expand_leaf(leaf, feature, condition)


  def expand_leaf(self, leaf, feature, condition):
    node = self.Node(feature, condition, leaf.x, leaf.y, leaf.parent, leaf.depth)

    self.leafs.append(node.left)
    self.leafs.append(node.right)

    if node.parent is None:
      self.root = node
    else:
      if node.parent.left is leaf:
        node.parent.left = node
      else:
        node.parent.right = node
    self.leafs.remove(leaf)


  def best_split(self, x, y):
    return sorted([self.one_feature_split(x, y, i) for i in range(x.shape[1])], key=lambda x: x[-1], reverse=True)[0]


  def one_feature_split(self, x, y, feature, k=10):
    sort_feature = x[:, feature]
    conditions = list()
    for _ in range(k):
      cond_ind = np.random.randint(len(sort_feature)-1)
      condition = (sort_feature[cond_ind] + sort_feature[cond_ind + 1]) / 2
      conditions.append((condition, self.information_gain(y, [y[sort_feature <= condition], y[sort_feature > condition]])))

    return feature, *sorted(conditions, key = lambda x: x[-1], reverse=True)[0]


  def score(self, x, y):
    out = np.array([self.predict(x_i) for x_i in x])
    return abs(y - out).sum() / len(y)


  class Node:
    def __init__(self, feature, condition, x,y, parent, depth=1):
      self.depth = depth
      self.parent = parent
      self.feature = feature
      self.condition = condition
      left = x[:, feature] <= condition
      right = x[:, feature] > condition
      self.left = DisicionTree.Leaf(x[left], y[left], self, self.depth + 1)
      self.right = DisicionTree.Leaf(x[right], y[right], self, self.depth + 1)


    def predict(self, x):
      if x[self.feature] <= self.condition:
        return self.left.predict(x)
      else:
        return self.right.predict(x)

  class Leaf:
    def __init__(self,x, y,parent, depth=1):
      self.entropy = entropy(y,base=2)
      self.parent = parent
      self.x, self.y = x, y
      self.depth = depth
      self.out = Counter(y).most_common()[0][0]
    def predict(self, x):
      return self.out





In [376]:
from sklearn.datasets import make_classification
x,y = make_classification(100,10)

In [377]:
x_t,y_t = x[1], y[0]
tree = DisicionTree(x,y)

In [378]:
tree.score(x,y)

0.5

In [379]:
tree.one_feature_split(x,y,0)

(0, 0.010607846136297838, 1.1257693834979818)

In [380]:
tree.best_split(x,y)

(0, -0.3455065757183338, 1.114339125643225)

In [364]:
tree.split_one_leaf()

In [368]:
Counter([1,2,3,3,3,3,1]).most_common()

[(3, 4), (1, 2), (2, 1)]

In [386]:
entropy([1,2,3,4],base=2)


1.8464393446710157

<font color = 'red' size = 5>Задание 3 </font>

Для каждого из подзаданий:

1. Проведите предварительную обработку данных.
2. Постройте модель классификации на основе метода решающего дерева из встроенной библиотеки.
3. Подберите оптимальные гиперпараметры модели используя различные оценки, кросс-валидацию и валидационные кривые.
4. Сделайте выводы о точности моделей.
5. Продумайте, как дерево может бороться с переобучением. Подвердите валидационными кривыми.
6. Оцените качество модели с помощью precision\recall score.

(https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html)



##### 3.1 Самый известный датасет в мире.

Используйте данные из сэта о [титанике](https://www.kaggle.com/c/titanic) для предсказания выживания.
Предврительно изучите и подготовьте данные.

#####  3.2 Предсказание диабета у пациентов.

Используйте данные из файла [diabetes.csv](https://www.kaggle.com/datasets/saurabh00007/diabetescsv) для предсказания исхода для пациентов. Столбец "outcome". Предварительно изучите и подготовьте данные.


##### 3.3 Данные о цветках ириса

[IrisDataset](https://www.kaggle.com/datasets/uciml/iris)

##### 3.4 Грибы

Поппробуйте предсказать съедобность гриба с помощью дерева принятия решений.
[mushrooms](https://www.kaggle.com/datasets/uciml/mushroom-classification/code)



<font color = 'red' size = 5>Задание 4 </font>

Используя встроенные методы реализуйте регрессию с помощью решающего дерева. Примените для данных из предыдущих ЛР. Сделайте выводы. По какому принципу работает дерево в случае регресии?